# NIRSpec IFU Data Reduction Pipeline

By the JWST ERS TEMPLATES team (B. Welch, T. Hutchison, +). 
Pre-release version, December 2023. 

Based on a combination of notebooks from STSCI:  
https://github.com/STScI-MIRI/MRS-ExampleNB/blob/main/Flight_Notebook1/MRS_FlightNB1.ipynb
https://github.com/spacetelescope/jwebbinar_prep/blob/main/spec_mode/spec_mode_stage_2.ipynb
https://github.com/spacetelescope/jwebbinar_prep/blob/main/spec_mode/spec_mode_stage_3.ipynb
https://github.com/spacetelescope/jwebbinar_prep/blob/main/ifu_session/jwebbinar5_nirspecifu.ipynb

Prerequisites: Install JWST pipeline. See TEMPLATES pipeline installation notebook (https://github.com/JWST-Templates/Notebooks/blob/main/0_install_pipeline.ipynb) for help.

# Example Pipeline Run on NIRSpec IFU Data

The output printed by the various pipeline stages is extensive -- which is great for troubleshooting and learning the pipeline but also makes these notebooks very large in size (which GitHub hates).  We've added an `stpipe-log.cfg` file to this repository that will pipe all of that printed output into a log file called `pipeline-run.log`, which will be generated in the same working directory.

----------

In [ ]:
# select target name for easier processing below
# MAKE SURE TARGET NAME MATCHES DATAFILE HEADERS!

target = 'SGAS1723'
#target = 'SGAS1226'
#target = 'SPT0418-47'
#target = 'SPT2147-50'

In [ ]:
import numpy as np
import glob
import os

# Modify the paths to the relevant directories on your machine
# ------------------------------------------------------------

# 1) point to where the jwst pipeline config files are located
home = "/Users/me/home/" # update with your home directory


# 2) point to where you keep your uncalibrated data (*uncal.fits)
if target == 'SGAS1723':
    input_path = os.path.join(home,'path/to/target/data/')
if target == 'SGAS1226':
    input_path = os.path.join(home,'path/to/target/data/')
if target == 'SPT0418-47':
    input_path = os.path.join(home,'path/to/target/data/')
if target == 'SPT2147-50':
    input_path = os.path.join(home,'path/to/target/data/')


# 3) point to where you want your processed outputs to live

# the pipeline will automatically use the most recent CRDS pmap reference file
pmap = 'pmap1105' # so, replace this value with that most recent version number

if target == 'SGAS1723':
    output_path = os.path.join(home,f'path/to/target/output/{pmap}/')
if target == 'SGAS1226':
    output_path = os.path.join(home,f'path/to/target/output/{pmap}/')
if target == 'SPT0418-47':
    output_path = os.path.join(home,f'path/to/target/output/{pmap}/')
if target == 'SPT2147-50':
    output_path = os.path.join(home,f'path/to/target/output/{pmap}/')

if os.path.exists(output_path) == False: # if folder doesn't exist
    print('Creating folder ' + output_path)
    os.system('mkdir ' + output_path) # creates the folder


###############################################
# Set up CRDS path and server environment variables
os.environ["CRDS_PATH"] = home + "crds_cache/jwst_ops"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"
###############################################


import zipfile
import urllib.request

import json

from astropy.io import fits
from astropy.utils.data import download_file
import astropy.units as u
from astropy import wcs
from astropy.wcs import WCS
from astropy.visualization import ImageNormalize, ManualInterval, LogStretch, LinearStretch, AsinhStretch

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
# The calwebb_spec and spec3 pipelines
from jwst.pipeline import Spec2Pipeline
from jwst.pipeline import Spec3Pipeline

# the level1 pipeline:
from jwst.pipeline import Detector1Pipeline

# data models
from jwst import datamodels

# association file utilities
from jwst.associations import asn_from_list as afl # Tools for creating association files
from jwst.associations.lib.rules_level2_base import DMSLevel2bBase # Definition of a Lvl2 association file
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base # Definition of a Lvl3 association file

In [ ]:
# Thou shalt know thy software version!
# We recommend updating your pipeline software regularly (pip install --upgrade jwst)
import jwst
print(jwst.__version__)

## Stage 1 - Detector-level processing
The first time the pipeline runs it will be glacially slow, as many GB of reference files need to be downloaded. With the CRDS environment variables set above, updated pmap reference files will be downloaded automatically each time the pipeline is run.


In [ ]:
# OPTIONAL: this a test to make sure that the files are accessible (useful troubleshooting)
# note that your program may name your TARGPROP differently

files = glob.glob(input_path + '*nrs*/*_uncal.fits') #list the uncalibrated (level 1b) files.
files = sorted(files)

ifu,sky = 0,0

for exposure in files:
    test = fits.open(exposure)
    head = test[0].header
    # NOTE the target name format can change for each observing program
    if head['TARGPROP'] == target+'-IFU': ifu+=1 
    elif head['TARGPROP'] == target+'-SKY': sky+=1
    elif head['TARGPROP'] == target+'-IFU-OFFSET': sky+=1
    else: print('not target, ', head["TARGPROP"])
    
print(f'Number of IFU: {ifu}, Number of sky: {sky}')

In [ ]:
# checking that the file system is in place for these data
# if not, creating the folders
folders_L2a = ['L2a/','L2a/sci/','L2a/bkg/']

for folder in folders_L2a:
    if os.path.exists(output_path + folder) == False: # if folder doesn't exist
        print('Creating folder ' + output_path + folder)
        os.system('mkdir ' + output_path + folder) # creates the folder


To control parameters directly from the notebook, the below version works. Simply add "det1.[step].[param] = [val]" before calling "det1(exposure)"
For example, to utilize multiple cores, add the line "det1.jump.maximum_cores = 'half'" and "det1.ramp_fit.maximum_cores = 'half'" to use half the available cores in those two steps.

Changing the maximum_cores variable in the parameter file can speed up this step. The default is "None", which uses one core. Other options are "quarter", "half", and "all", which will use 1/4, 1/2, or all available cores respectively. We find that "half" provides good performance while still allowing your computer to perform other simple tasks.

In previous pipeline versions, the "expand_large_events" flag for the jump detection step was turned off by default. It is now recommended that this flag be set to True to better remove cosmic ray snowballs.

In [ ]:
# Run the pipeline, splitting outputs into "sci" and "bkg" output folders
# If "leak calibration" exposures were taken, they will be in the same folders 
# as their associated observations

files = glob.glob(input_path + '*nrs*/*_uncal.fits') #list the uncalibrated (level 1b) files.
files = sorted(files)

for exposure in files: 
    det1 = Detector1Pipeline()
    # set output directory based on sci vs bkg exposure
    head = fits.open(exposure)[0].header
    if head['TARGPROP'] == target+'-IFU':
        det1.output_dir = output_path + 'L2a/sci'
    elif (head['TARGPROP'] == target+'-SKY') or (head['TARGPROP'] == target+'-IFU-OFFSET'):
        det1.output_dir = output_path + 'L2a/bkg'
    else: print('not target')
    # now we set other parameters:
    det1.save_results = True
    det1.jump.maximum_cores = 'half'
    det1.jump.expand_large_events = True
    det1.ramp_fit.maximum_cores = 'half'

    det1(exposure)


### Correct 1/f noise
The NIRSpec detectors are read-noise limited. To correct this read noise, we utilize the NSClean package (B. Rauscher, https://ui.adsabs.harvard.edu/abs/2023arXiv230603250R/abstract). Basically this fits the detector noise in Fourier space using unilluminated pixels set by a user-defined mask. We build the mask using pre-existing cal.fits files (can be downloaded from MAST).  

Here we only correct the science exposures, as that is all we plan to use going forward. Our current understanding is that the leakcals and background exposures only serve to add noise (see TEMPLATES overview paper for more details: ***LINK***). At least for TEMPLATES, the lack of bright stars on the closed MSA means the leakcals aren't really providing any improvement. And the background exposures almost perfectly match the predicted backgrounds. So we ignore the leakcals, and subtract the expected background after the pipeline has been run. 

In [ ]:
# make "destriped" L2a files
folders_L2a_destripe = ['L2a_destripe','L2a_destripe/sci/','L2a_destripe/bkg/']

for folder in folders_L2a_destripe:
    if os.path.exists(output_path + folder) == False: # if folder doesn't exist
        print('Creating folder ' + output_path + folder)
        os.system('mkdir ' + output_path + folder) # creates the folder


NSClean requires that all science pixels be properly masked, so that only the detector noise is included in the fit. To do this, we utilize pre-existing stage 2 pipeline products (cal.fits files), which mask out everything but the science pixels. These can be downloaded from MAST if needed. We have experimented with masking pixels flagged as snowballs, but we have found that this leads to artifacts in the output files from NSClean (likely because not enough pixels are available in certain columns to get a good fit). We thus have commented out this part of the mask making code. Large snowballs can be masked manually if they are found to be problematic. The central rows containing the fixed slits are also masked.

This code was largely written by Ian Wong, modified to work with NSClean by B. Welch. 

In [ ]:
# make masks using existing cal.fits files:
import scipy.ndimage as nd

def make_mask(ratefiles, calfiles, output_dir=None):
    sortrate = sorted(ratefiles)
    sortcal = sorted(calfiles)
    masks = []
    for i, fi in enumerate(sortrate):
        hdu = fits.open(fi)
        dq = hdu['DQ',1].data
        spec_dq = dq == 4 # == 4 masks all jump detections. Use == 0 to only use pixels with no data quality flags
        cal = sortcal[i]
        # make initial mask from cal file
        spec_mask = ~np.isnan(fits.getdata(cal))
        # create a bit of a buffer
        spec_mask = nd.binary_dilation(spec_mask, iterations=2)
        # Also mask out fixed slit region at center of detectors
        spec_mask[900:1100,:] = True
        #full_mask = np.logical_or(spec_mask,spec_dq) # commented out if not using DQ flags in automated masking step
        full_mask = ~spec_mask #full_mask
        
        if output_dir:
            hdu = fits.PrimaryHDU(full_mask.astype(float))
            hdu.writeto(os.path.join(output_dir, fi.split("/")[-1].replace('rate','mask2')), overwrite=True)
        else:
            output_dir = os.path.dirname(fi)
            hdu = fits.PrimaryHDU(full_mask.astype(float))
            hdu.writeto(os.path.join(output_dir, fi.split("/")[-1].replace('rate','mask2')), overwrite=True)
            output_dir = None

        masks.append(full_mask)
    return masks


# select rate/cal files to use - probably in a different directory
if target == 'SGAS1723':
    ratefiles = sorted(glob.glob(output_path + folders_L2a[2] + 'jw01355028001_0[2,6]101*rate.fits'))
    caldir = 'path/to/existing/cal/files/' # update with your directory
if target == 'SGAS1226':
    ratefiles = sorted(glob.glob(output_path + folders_L2a[0] + 'sci/jw*_02101*rate.fits', recursive=True))
    caldir = 'path/to/existing/cal/files/' # update with your directory
if target == 'SPT0418-47':
    ratefiles = sorted(glob.glob(output_path + folders_L2a[0] + 'sci/jw*_02101*nrs1_rate.fits', recursive=True))
    caldir = 'path/to/existing/cal/files/' # update with your directory
if target == 'SPT2147-50':
    ratefiles = sorted(glob.glob(output_path + folders_L2a[0] + 'sci/jw*_02101*nrs1_rate.fits', recursive=True))
    caldir = 'path/to/existing/cal/files/' # update with your directory

calfiles = sorted(glob.glob(caldir + '**/*cal.fits', recursive=True))
maskdir = os.path.join(output_path, 'destripe_masks/')
if os.path.exists(maskdir) == False: # if folder doesn't exist
    os.system('mkdir ' + maskdir) # creates the folder

masks = make_mask(ratefiles, calfiles, output_dir=None) 


*Setting up NSClean & importing the package*:  

We recommend downloading the NSClean code [from the JWST "Publications for Scientists" website](https://webb.nasa.gov/content/forScientists/publications.html), placing it somewhere that you can point to, & unzipping the file.  We'll manually import the package in the following cell.

In [ ]:
# NSClean Setup:
# Configuration needs to come first
os.environ['MKL_NUM_THREADS'] = '8'   # Enable multi-threading
os.environ['NSCLEAN_USE_CUPY'] = 'NO' # Use GPU. If you change this, you will
                                         #   need to restart python. In practice,
                                         #   I have found that most of the time
                                         #   the GPU is no faster than using CPUs.

# Import the appropriate numerical libraries
if os.getenv('NSCLEAN_USE_CUPY') == 'YES':
    import cupy as cp
    import numpy as np
else:
    import numpy as cp
    import numpy as np


# importing NSClean package
import sys 
sys.path.append('/Users/me/path/to/NSClean/') # to import the nsclean module faster
from nsclean import nsclean as nc

In [ ]:
##################
# NRS1 FILE DESTRIPING
##################

if target == 'SGAS1723':
    # NOTE this only checks the science directory L2a/sci
    files = sorted(glob.glob(output_path + folders_L2a[1] + 'jw*_0[2,6]*_nrs1_rate.fits', recursive=True))
else:
    files = sorted(glob.glob(output_path + folders_L2a[1] + 'jw*_02101*_nrs1_rate.fits', recursive=True))

maskfiles = sorted(glob.glob(output_path+ folders_L2a[0] + '**/*nrs1_mask2.fits', recursive=True))


for i in np.arange(len(files)):
    print(f'starting file {i}')
    
    # Download FITS file
    hdul = fits.open(files[i])
    H0 = hdul[0].header
    H1 = hdul[1].header
    D1 = cp.array(hdul[1].data)
    #hdul.close()
    
    D1[np.isnan(D1)] = 0.0 # cleaner doesn't like nans, so set them to zero
    
    # Instantiate an NSClean object
    M = fits.getdata(maskfiles[i]).astype(bool)
    cleaner = nc.NSClean('NRS1', M)

    # Clean it
    D1 = cleaner.clean(D1, buff=True)
    
    # Save it to FITS
    H0['comment'] = 'Processed by NSClean Rev. '+nc.__version__
    if os.getenv('NSCLEAN_USE_CUPY') == 'YES':
        hdul[1].data = cp.asnumpy(D1)
    else:
        hdul[1].data = D1
    
    
    # final version
    hdul.writeto(output_path + folders_L2a_destripe[1] + \
                     nc.chsuf(os.path.basename(files[i]), '.cln_mask.fits'),
                        overwrite=True)
    hdul.close()
    print(f'Done file {i}')

In [ ]:
##################
# NRS2 FILE DESTRIPING
##################

if target == 'SGAS1723':
    # NOTE this only checks the science directory L2a/sci
    files = sorted(glob.glob(output_path + folders_L2a[1] + 'jw*_0[2,6]*_nrs2_rate.fits', recursive=True))
else:
    files = sorted(glob.glob(output_path + folders_L2a[1] + 'jw*_02101*_nrs2_rate.fits', recursive=True))

maskfiles = sorted(glob.glob(output_path+ folders_L2a[0] + '**/*nrs2_mask2.fits', recursive=True))

# Instantiate an NSClean object
cleaner = nc.NSClean('NRS2', M)

for i in np.arange(len(files)):
    print(f'starting file {i}')
        
    # Download FITS file
    hdul = fits.open(files[i])
    H0 = hdul[0].header
    H1 = hdul[1].header
    D1 = cp.array(hdul[1].data)
    #hdul.close()
    
    D1[np.isnan(D1)] = 0.0 # cleaner doesn't like nans, so set them to zero
    
    # Instantiate an NSClean object
    M = fits.getdata(maskfiles[i]).astype(bool)
    cleaner = nc.NSClean('NRS2', M)

    # Clean it
    D1 = cleaner.clean(D1, buff=True)
    
    # Save it to FITS
    H0['comment'] = 'Processed by NSClean Rev. '+nc.__version__
    if os.getenv('NSCLEAN_USE_CUPY') == 'YES':
        hdul[1].data = cp.asnumpy(D1)
    else:
        hdul[1].data = D1

    # final version
    hdul.writeto(output_path + folders_L2a_destripe[1] + \
                     nc.chsuf(os.path.basename(files[i]), '.cln_mask.fits'),
                        overwrite=True)
    hdul.close()
    print(f'Done file {i}')

## Stage 2 - Produce calibrated exposures
These next two cells assume that you have two directories with the L2a outputs from the Stage 1 pipeline as defined above. We have named these "sci" for the science observations, and "bkg" for the background observations.  

As the pipeline is set up here, no background subtraction is performed in this stage. 

This is the stage where leak calibration exposures can be used to remove any stray light leaking through the MSA onto the detectors. We do not currently apply that correction, as it simply introduces additional noise and we do not find significant light leakage in our observations. We have included code previously used to do this leakcal subtraction, however we will not be supporting that code.

In [ ]:
# checking that the file system is in place for these data
# if not, creating the folders
folders_L2b = ['L2b_destripe/','L2b_destripe/sci/','L2b_destripe/bkg/']

for folder in folders_L2b:
    if os.path.exists(output_path + folder) == False: # if folder doesn't exist
        print('Creating folder ' + output_path + folder)
        os.system('mkdir ' + output_path + folder) # creates the folder


To include leakcals (and background subtraction if desired) in the Stage 2 pipeline, an association file is needed. Below is a function to create these asn files from a combination of science, leak, and background exposures.

In [ ]:
# first, lets adapt the writel3asn function to produce what we want for the level 2 pipeline
def writel2asn(scifiles, leakfiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMSLevel2bBase, product_name=prodname)
    
    # Add leakcal files to the association
    if leakfiles:
        for ii in range(len(leakfiles)):
            asn['products'][0]['members'].append({'expname': leakfiles[ii], 'exptype': 'imprint'})
            
    # Add background files to the association
    if bgfiles:
        nbg=len(bgfiles)
        for ii in range(0,nbg):
            asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

For TEMPLATES, we have found that the leakcals (observations taken with MSA/IFU all closed) only introduce noise into our final products. TEMPLATES are all fairly dark fields (not many bright stars nearby) so leakcals only add detector noise, and do not remove any real leaking light. We thus do not process leakcals (use_leak flag set to False). If you wish to subtract leakcals, set use_leak = True. This will subtract leakcals on a dither-by-dither basis, assuming each science dither includes a corresponding leakcal. We leave a commented-out block of code at the end which creates an average leakcal to subtract from science exposures. We do not commit to maintain this version, but leave it for reference.

We have also found that subtracting backgrounds using specific background pointings introduces more noise than we would prefer. We therefore subtract the background using the JWST backgrounds tool after the Stage 3 pipeline has been run. If you would prefer to use dedicated background pointings (e.g. if you are observing below 1 micron, for which the background tool has some known errors), set the process_background flag to True.

Below is a block of code that creates the appropriate asn files for each exposure. This involves a lot of tedious file grouping, leading to the large and somewhat messy-looking code blocks. We have tried to include many comments to clarify the code. 

In [ ]:
# this has become a huge mess and I'm sorry


# "dither" leak_version will use each leakcal with its associated sci exposure
# "all" will average all leakcals and subtract the average from each sci exposure
leak_version = 'dither' # 'dither' or 'all'
use_leak = False # actually use leakcals?
############


process_background = False # actually process backgrounds??
#############


# define some path names for easy access
level1_sci_dir = os.path.join(output_path, folders_L2a_destripe[1]) 
level1_bg_dir = os.path.join(output_path, folders_L2a_destripe[2]) 
#leakdir_sci = os.path.join(output_path, 'L2a/leaksci')
#leakdir_bg = os.path.join(output_path, 'L2a/leakbkg')
level2outputdir = os.path.join(output_path, folders_L2b[0])

# define variables for file naming 
# user will have to look these up and edit them

program_id = '01355' # 5-digit string. ID of observing program. '01355' for TEMPLATES
if target == 'SGAS1723':
    obs_num_sci = ['007','027'] # 3-digit string. Observation number for science observations. 
    # This is a List for 1723 b/c we are using failed observations where possible
    obs_num_bkg = '028' # 3-digit string. Observation number for background observations. 
    visit_num = '001' # 3-digit string. Visit number. '001' for most observations
    group_sci = ['02101', '06101'] # List of 5-digit strings. Visit group numbers for science observations. 
    # For S1723, '02' is g140h observations, '06' is g395h observations
    group_leak = ['04101', '08101'] # List of 5-digit strings. Visit group numbers for leakcals. 
    # For S1723, '04' and '08' are dedicated leakcals
    # Group numbers should be findable in the observation APT file
    
if target == 'SGAS1226':
    obs_num_sci = '001' # 3-digit string. Observation number for science observations. 
    obs_num_bkg = '002' # 3-digit string. Observation number for background observations. 
    visit_num = '001' # 3-digit string. Visit number. '001' for most observations
    group_sci = ['02101'] # List of 5-digit strings. Visit group numbers for science observations. 
    group_leak = ['02103'] # List of 5-digit strings. Visit group numbers for leakcals. 
    group_leak_bg = ['04101']
    # for 1226, the leakcals have different group numbers for the off-target pointing. No idea why. 'Tis silly.

if target == 'SPT0418-47':
    obs_num_sci = '011'
    obs_num_bkg = '012'
    visit_num = '001'
    group_sci = ['02101']
    group_leak = ['02103']
    
if target == 'SPT2147-50':
    obs_num_sci = '019'
    obs_num_bkg = '020'
    visit_num = '001'
    group_sci = ['02101']
    group_leak = ['04101']

Take care when defining all of these files. Getting the wrong science, background, and leakcal exposures wrong can really mess up the final product. We print the output association files, and recommend checking the contents of these files prior to running the Spec2 pipeline.

In [ ]:
if target == 'SGAS1723':
    endfile = 'rate.cln_mask' 
    # and combine some of these for convenience:
    sci_base = 'jw' + program_id + obs_num_sci[0] + visit_num + '_' # I always feel like
    bkg_base = 'jw' + program_id + obs_num_bkg + visit_num + '_' # somebody's watching meeee
    # second set of failed observations gets defined separately
    sci_base2 = 'jw' + program_id + obs_num_sci[1] + visit_num + '_' 

    # specify science files
    # currently assumes we use 2 gratings. Will need to edit for more/fewer grating observations
    scifiles_g1 = sorted(glob.glob(level1_sci_dir + sci_base + group_sci[0] + '*'+endfile+'.fits'))
    # add in second set of failed observations
    scifiles_g1 = sorted(scifiles_g1 + glob.glob(level1_sci_dir + sci_base2 + group_sci[0] + '*'+endfile+'.fits'))
    scifiles_g2 = sorted(glob.glob(level1_sci_dir + sci_base2 + group_sci[1] + '*'+endfile+'.fits'))

    # and background files:
    bgfiles_g1 = sorted(glob.glob(level1_bg_dir + bkg_base + group_sci[0] + '*'+endfile+'.fits'))
    bgfiles_g2 = sorted(glob.glob(level1_bg_dir + bkg_base + group_sci[1] + '*'+endfile+'.fits'))

    # collect leakcal files
    sci_leakfiles_g1_all = sorted(glob.glob(level1_sci_dir + sci_base + group_leak[0] + '*'+endfile+'.fits'))
    sci_leakfiles_g2_all = sorted(glob.glob(level1_sci_dir + sci_base + group_leak[1] + '*'+endfile+'.fits'))

    bg_leakfiles_g1_all = sorted(glob.glob(level1_bg_dir + bkg_base + group_leak[1] + '*'+endfile+'.fits'))
    bg_leakfiles_g2_all = sorted(glob.glob(level1_bg_dir + bkg_base + group_leak[1] + '*'+endfile+'.fits'))

    filemin, filemax = -17, -10
    if endfile != 'rate':
        diff = len('rate') - len(endfile)
        filemin += diff
        filemax += diff 
 
    # this version assumes you have the same number of dithers for science, leak, and background observations
    if leak_version == 'dither':
        for i,file in enumerate(scifiles_g1):
            #leak = sci_leakfiles_g1_all[i]
            expnum = file[filemin:filemax]
            if i < 8:
                outfile = 'jw'+program_id+'-o'+obs_num_sci[0]+'_'+group_sci[0]+'_spec2_'+expnum+'_asn.json' 
            else:
                outfile = 'jw'+program_id+'-o'+obs_num_sci[1]+'_'+group_sci[0]+'_spec2_'+expnum+'_asn.json'
            outfile = os.path.join(level2outputdir, outfile)
            if use_leak: writel2asn([file], [leak], None, outfile, 'Level2') 
            else: writel2asn([file], None, None, outfile, 'Level2') 
            print(outfile)

        for i,file in enumerate(scifiles_g2): # comment out this loop if only one grating exists
            #leak = sci_leakfiles_g2_all[i] 
            expnum = file[filemin:filemax]
            outfile = 'jw'+program_id+'-o'+obs_num_sci[1]+'_'+group_sci[1]+'_spec2_'+expnum+'_asn.json'
            outfile = os.path.join(level2outputdir, outfile)
            if use_leak: writel2asn([file], [leak], None, outfile, 'Level2') 
            else: writel2asn([file], None, None, outfile, 'Level2') 
            print(outfile)

        if process_background == True:
            for i,file in enumerate(bgfiles_g1):
                #leak = bg_leakfiles_g1_all[i]
                expnum = file[filemin:filemax]
                outfile = 'jw'+program_id+'-o'+obs_num_bkg+'_'+group_sci[0]+'_spec2_'+expnum+'_asn.json'
                outfile = os.path.join(level2outputdir, outfile)
                if use_leak: writel2asn([file], [leak], None, outfile, 'Level2') 
                else: writel2asn([file], None, None, outfile, 'Level2') 
                print(outfile)

            for i,file in enumerate(bgfiles_g2): # comment out this loop if only one grating exists
                #leak = bg_leakfiles_g2_all[i]
                expnum = file[filemin:filemax]
                outfile = 'jw'+program_id+'-o'+obs_num_bkg+'_'+group_sci[1]+'_spec2_'+expnum+'_asn.json'
                outfile = os.path.join(level2outputdir, outfile)
                if use_leak: writel2asn([file], [leak], None, outfile, 'Level2') 
                else: writel2asn([file], None, None, outfile, 'Level2') 
                print(outfile)

    print()

In [ ]:
if target == 'SGAS1226':
    endfile = 'rate.cln_mask'
    # and combine some of these for convenience:
    sci_base = 'jw' + program_id + obs_num_sci + visit_num + '_' # I always feel like
    bkg_base = 'jw' + program_id + obs_num_bkg + visit_num + '_' # somebody's watching meeee

    # specify science files
    scifiles_g1 = sorted(glob.glob(level1_sci_dir + sci_base + group_sci[0] + '*rate.cln_mask.fits'))

    # and background files:
    bgfiles_g1 = sorted(glob.glob(level1_bg_dir + bkg_base + group_sci[0] + '*rate.cln_mask.fits'))

    # collect leakcal files
    sci_leakfiles_g1_all = sorted(glob.glob(level1_sci_dir + sci_base + group_leak[0] + '*rate.cln_mask.fits'))
    # because SGAS1226 has different numbers, we use the group_leak_bg variable here
    bg_leakfiles_g1_all = sorted(glob.glob(level1_bg_dir + bkg_base + group_leak_bg[0] + '*rate.cln_mask.fits'))

    filemin, filemax = -17, -10
    if endfile != 'rate':
        diff = len('rate') - len(endfile)
        filemin += diff
        filemax += diff 

    # this version assumes you have the same number of dithers for science, leak, and background observations
    if leak_version == 'dither':
        for i,file in enumerate(scifiles_g1):
            #leak = sci_leakfiles_g1_all[i]
            expnum = file[filemin:filemax]
            outfile = 'jw'+program_id+'-o'+obs_num_sci+'_'+group_sci[0]+'_spec2_'+expnum+'_asn.json' 
            outfile = os.path.join(level2outputdir, outfile)
            if use_leak: writel2asn([file], [leak], None, outfile, 'Level2') 
            else: writel2asn([file], None, None, outfile, 'Level2') 
            print(outfile)

        if process_background == True:
            for i,file in enumerate(bgfiles_g1):
                #leak = bg_leakfiles_g1_all[i]
                expnum = file[filemin:filemax]
                outfile = 'jw'+program_id+'-o'+obs_num_bkg+'_'+group_sci[0]+'_spec2_'+expnum+'_asn.json'
                outfile = os.path.join(level2outputdir, outfile)
                if use_leak: writel2asn([file], [leak], None, outfile, 'Level2') 
                else: writel2asn([file], None, None, outfile, 'Level2') 
                print(outfile)


    print()

In [ ]:
if (target == 'SPT0418-47') or (target == 'SPT2147-50'):
    endfile = 'rate.cln_mask'
    # and combine some of these for convenience:
    sci_base = 'jw' + program_id + obs_num_sci + visit_num + '_' # I always feel like
    bkg_base = 'jw' + program_id + obs_num_bkg + visit_num + '_' # somebody's watching meeee

    # specify science files
    scifiles_g1 = sorted(glob.glob(level1_sci_dir + sci_base + group_sci[0] + '*nrs1_rate.cln_mask.fits'))

    # and background files:
    bgfiles_g1 = sorted(glob.glob(level1_bg_dir + bkg_base + group_sci[0] + '*nrs1_rate.cln_mask.fits'))

    # collect leakcal files
    sci_leakfiles_g1_all = sorted(glob.glob(level1_sci_dir + sci_base + group_leak[0] + '*nrs1_rate.cln_mask.fits'))
    # because SGAS1226 has different numbers, we use the group_leak_bg variable here
    bg_leakfiles_g1_all = sorted(glob.glob(level1_bg_dir + bkg_base + group_leak[0] + '*nrs1_rate.cln_mask.fits'))

    filemin, filemax = -17, -10
    if endfile != 'rate':
        diff = len('rate') - len(endfile)
        filemin += diff
        filemax += diff 

    # this version assumes you have the same number of dithers for science, leak, and background observations
    if leak_version == 'dither':
        for i,file in enumerate(scifiles_g1):
            #leak = sci_leakfiles_g1_all[i]
            expnum = file[filemin:filemax]
            outfile = 'jw'+program_id+'-o'+obs_num_sci+'_'+group_sci[0]+'_spec2_'+expnum+'_asn.json' 
            outfile = os.path.join(level2outputdir, outfile)
            if use_leak: writel2asn([file], [leak], None, outfile, 'Level2') 
            else: writel2asn([file], None, None, outfile, 'Level2') 
            print(outfile)

        if process_background == True:
            for i,file in enumerate(bgfiles_g1):
                #leak = bg_leakfiles_g1_all[i]
                expnum = file[filemin:filemax]
                outfile = 'jw'+program_id+'-o'+obs_num_bkg+'_'+group_sci[0]+'_spec2_'+expnum+'_asn.json'
                outfile = os.path.join(level2outputdir, outfile)
                if use_leak: writel2asn([file], [leak], None, outfile, 'Level2') 
                else: writel2asn([file], None, None, outfile, 'Level2') 
                print(outfile)


    print()

In [ ]:
# the below version will use average of all leakcals, and subtract that from each science exposure
# we are not supporting this currently, but leave it in case anyone wants to attempt to resurrect it

'''
leakfiles_04_nrs1 = sorted(glob.glob(leakdir_sci + '/jw01355027001_04101_*nrs1_rate.fits'))
leakfiles_04_nrs2 = sorted(glob.glob(leakdir_sci + '/jw01355027001_04101_*nrs2_rate.fits'))
leakfiles_08_nrs1 = sorted(glob.glob(leakdir_sci + '/jw01355027001_08101_*nrs1_rate.fits'))
leakfiles_08_nrs2 = sorted(glob.glob(leakdir_sci + '/jw01355027001_08101_*nrs2_rate.fits'))
bg_leakfiles_04_nrs1 = sorted(glob.glob(leakdir_bg + '/jw01355028001_04101_*nrs1_rate.fits'))
bg_leakfiles_04_nrs2 = sorted(glob.glob(leakdir_bg + '/jw01355028001_04101_*nrs2_rate.fits'))
bg_leakfiles_08_nrs1 = sorted(glob.glob(leakdir_bg + '/jw01355028001_08101_*nrs1_rate.fits'))
bg_leakfiles_08_nrs2 = sorted(glob.glob(leakdir_bg + '/jw01355028001_08101_*nrs2_rate.fits'))


if leak_version == 'all':
    for file in scifiles_02:
        if "nrs1" in file:
            leak = leakfiles_04_nrs1
        elif "nrs2" in file:
            leak = leakfiles_04_nrs2
        else:
            print('Check file names - no detector label found')
            leak = None
        expnum = file[-17:-10]
        outfile = 'jw01355-o027_02101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], leak, None, outfile, 'Level2')
        print(outfile)

    for file in scifiles_06:
        if "nrs1" in file:
            leak = leakfiles_08_nrs1
        elif "nrs2" in file:
            leak = leakfiles_08_nrs2
        else:
            print('Check file names - no detector label found')
            leak = None
        expnum = file[-17:-10]
        outfile = 'jw01355-o027_06101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], leak, None, outfile, 'Level2')
        print(outfile)
        
    for file in bgfiles_02:
        if "nrs1" in file:
            leak = leakfiles_04_nrs1
        elif "nrs2" in file:
            leak = leakfiles_04_nrs2
        else:
            print('Check file names - no detector label found')
            leak = None
        expnum = file[-17:-10]
        outfile = 'jw01355-o028_02101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], leak, None, outfile, 'Level2')
        print(outfile)

    for file in bgfiles_06:
        if "nrs1" in file:
            leak = leakfiles_08_nrs1
        elif "nrs2" in file:
            leak = leakfiles_08_nrs2
        else:
            print('Check file names - no detector label found')
            leak = None
        expnum = file[-17:-10]
        outfile = 'jw01355-o028_06101_spec2_'+expnum+'_asn.json'
        outfile = os.path.join(level2outputdir, outfile)
        writel2asn([file], leak, None, outfile, 'Level2')
        print(outfile)
'''

In [ ]:
# And now we run the pipeline with the asn files we just made! 

if target == 'SGAS1723':
    asnfiles_sci = glob.glob(level2outputdir+'jw'+program_id+'-o'+obs_num_sci[0]+'*asn.json') 
    asnfiles_sci += glob.glob(level2outputdir+'jw'+program_id+'-o'+obs_num_sci[1]+'*asn.json')
else:
    asnfiles_sci = glob.glob(level2outputdir+'jw'+program_id+'-o'+obs_num_sci+'*asn.json')
asnfiles_bkg = glob.glob(level2outputdir+'jw'+program_id+'-o'+obs_num_bkg+'*asn.json')

for asn in asnfiles_sci:
    spec2 = Spec2Pipeline()
    spec2.output_dir = output_path + folders_L2b[1]
    spec2.save_results = True
    spec2(asn)

if process_background == True:
    for asn in asnfiles_bkg:
        spec2 = Spec2Pipeline()
        spec2.output_dir = output_path + folders_L2b[2]
        spec2.save_results = True
        spec2(asn)

## Stage 3 - Creating final data cubes
This stage compiles the individual calibrated exposures into a single, (theoretically) science-ready data cube. We first have to define an association file, which tells the pipeline which files to use as science exposures, and which files to use as backgrounds.

As mentioned above, for TEMPLATES we are subtracting a smooth, calculated background after Stage 3 processing. If instead you wish to subtract an observed background, set the process_background flag above to True to include background pointings in the Stage 3 association file. 

#### NOTE on outlier detection
When we started creating this notebook, outlier_detection was dysfunctional. T. Hutchison therefore developed our own sigma clipping routine (in a separate notebook). The pipeline outlier_detection step has now been fixed (versions 1.11.3 and later), however it may require some parameter tuning to give the best results. We have found that the best performance comes from using the updated pipeline outlier detection step alongside our custom routine (see Hutchison et al. for details: ***LINK***). We also include a first-pass cut on the Stage 2 files to remove the most egregious outliers, but most of the final cleaning comes in the outlier detection step in the pipeline (and then the post-processing algorithm from Hutchison et al.).

In [ ]:
# checking that the file system is in place for these data
# if not, creating the folders
folders_L3 = ['L3_destripe/']

for folder in folders_L3:
    if os.path.exists(output_path + folder) == False: # if folder doesn't exist
        print('Creating folder ' + output_path + folder)
        os.system('mkdir ' + output_path + folder) # creates the folder


In [ ]:
# BELOW COPIED FROM DAVID LAW'S MIRI MRS NOTEBOOK: 
# https://github.com/STScI-MIRI/MRS-ExampleNB/blob/main/Flight_Notebook1/MRS_FlightNB1.ipynb
# 
# Define a useful function to write out a Lvl3 association file from an input list
# Note that any background exposures have to be of type x1d.
def writel3asn(scifiles, bgfiles, asnfile, prodname):
    # Define the basic association of science files
    asn = afl.asn_from_list(scifiles, rule=DMS_Level3_Base, product_name=prodname)
        
    # Add background files to the association
    if bgfiles:
        nbg=len(bgfiles)
        for ii in range(0,nbg):
            asn['products'][0]['members'].append({'expname': bgfiles[ii], 'exptype': 'background'})
        
    # Write the association to a json file
    _, serialized = asn.dump()
    with open(asnfile, 'w') as outfile:
        outfile.write(serialized)

In [ ]:
# COURTESY OF D. LAW:
# Manually remove most egregious outliers from the cal.fits files

from stcal import dqflags


def cut_cal(infile, outfile, max_threshold=20, min_threshold=-1):
    hdu=fits.open(infile)
    sci=hdu['SCI'].data
    dq=hdu['DQ'].data
    
    dnubit=dqflags.interpret_bit_flags('DO_NOT_USE', mnemonic_map=datamodels.dqflags.pixel)
    indx=np.where((dq & dnubit) != 0)
    sci[indx]=np.nan

    indx=np.where((sci > max_threshold) | (sci < min_threshold))
    sci[indx]=np.nan
    dq[indx] = np.bitwise_or(dq[indx], dnubit)
    
    hdu['SCI'].data=sci
    hdu.writeto(outfile, overwrite=True)
    

orig_calfiles = glob.glob(level2outputdir + '/sci/*cal.fits') # cal files output from L2 pipeline

# set different thresholds for each target - done by-eye w/ L2 ratefiles
if target == 'SGAS1723': 
    calmax_blue = 2000
    calmax_red = 350
    calmax = 2000
#if target == 'SGAS1723': calmax = 350 # second cut for redder grating
if target == 'SGAS1226': calmax = 50
if target == 'SPT0418-47': calmax = 40
if target == 'SPT2147-50': calmax = 30

calfiles = glob.glob(level2outputdir + '/sci/*cal.fits') # cal files output from L2 pipeline

for file in calfiles:
    outfile = file[:-5] + '2.fits'
    cut_cal(file, outfile, max_threshold=calmax, min_threshold=-10)
    print(outfile)

bkg_calfiles = glob.glob(level2outputdir + '/bkg/*cal.fits') # cal files output from L2 pipeline
if process_background == True:
    for file in bkg_calfiles:
        outfile = file[:-5] + '2.fits'
        cut_cal(file, outfile, max_threshold=calmax, min_threshold=-10)
        print(outfile)

In [ ]:
# Make association files 
calfiles = glob.glob(output_path + folders_L2b[1] + '*cal2.fits')

bkgfiles_cal = glob.glob(output_path + folders_L2b[2] + '*cal2.fits') 

# We have included several versions to generate Spec3 association files
# 'nobg' runs only the science observations, without subtracting any background (This is what we recommend)
# 'bgonly' was set up to only process the background pointings, primarily as an internal test
# 'standard' implements the pipeline's background subtraction, which we do NOT recommend using
version = 'nobg' 

if version == 'standard':
    if len(folders_L3) == 1:
        fold = folders_L3[0]
        asnfile = os.path.join(output_path, fold, 'L3asn.json')
    else:
        fold = folders_L3[1]
        asnfile = os.path.join(output_path, fold, 'L3asn.json')
    lev3asnname = 'Level3_' + target + '_BGSUB'
    writel3asn(calfiles, bkgfiles, asnfile, lev3asnname) 
    
if version == 'bgonly':
    fold = folders_L3[0]
    asnfile = os.path.join(output_path, fold, 'L3asn.json')
    lev3asnname = 'Level3_' + target + '_BGONLY'
    writel3asn(bkgfiles_cal, None, asnfile, lev3asnname)
    
if version == 'nobg':
    if len(folders_L3) == 1:
        fold = folders_L3[0]
        asnfile = os.path.join(output_path, fold, 'L3asn.json')
    else:
        fold = folders_L3[3]
        asnfile = os.path.join(output_path, fold, 'L3asn.json')
    lev3asnname = 'Level3_' + target + '_NOBG'
    writel3asn(calfiles, None, asnfile, lev3asnname)
   

In [ ]:
# Next step, call stage 3 pipeline on new asn file

# setting it up this way (rather than with "call") gives a bit more flexibility to edit parameters on the fly
spec3 = Spec3Pipeline()
spec3.output_dir = output_path + fold # use folder defined in cell above - output to same dir as asn file
#spec3.logcfg = spec3.output_dir + 'test-log.cfg'

# To skip the outlier detection step, uncomment this line
# We recommend using both the outlier detection step AND the custom 
# sigma clipping algorithm described in Hutchison et al.
#spec3.outlier_detection.skip = True 

# cube building parameters:
#spec3.cube_build.coord_system = 'ifualign' # Stay in IFU coordinates, don't rotate to N=up E=left coordinates
# For some programs, it may be desireable to drizzle to finer pixel scales. The below commands can do that.
#spec3.cube_build.scale1 = 0.075 # Final pixel scale, in arcsec
#spec3.cube_build.scale2 = 0.075

spec3.save_results = True # DON'T FORGET THIS OR YOU'LL WASTE SEVERAL HOURS FOR NAUGHT!

spec3.run(asnfile)

## Last Step: *DIY background subtraction!*  

The background pointings generally just introduce additional noise, so we are instead subtracting the expected backgrounds calculated from the JWST background tool (https://jwst-docs.stsci.edu/jwst-other-tools/jwst-backgrounds-tool). This will make the final result a bit cleaner, and the measured backgrounds match the expected backgrounds well for each TEMPLATES target (based on testing we have done). 

In [ ]:
from scipy.interpolate import interp1d

def exp_bg_sub(cubefile, bgfile, outfile=None):
    # first, load the expected background data:
    exp = np.loadtxt(bgfile)
    expwl = np.array([line[0] for line in exp])
    exptot = np.array([line[1] for line in exp])
    # and make an interp1d object sdo we can get the background at any given point:
    bginterp = interp1d(expwl, exptot)
    # now load the data cube:
    hdu = fits.open(cubefile)
    data = hdu[1].data
    head1 = hdu[1].header
    cubewl = np.arange(head1['CRVAL3'],
                       head1['CRVAL3']+(head1['CDELT3']*len(data)),
                       head1['CDELT3'])
    if len(cubewl) > len(data):
        cubewl = cubewl[:-1]
    # next, evaluate background on data wavelength points:
    bgcalculated = bginterp(cubewl)
    bgcube = np.tile(bgcalculated, (data.shape[2],data.shape[1],1)).T # cubeify!
    # and subtract!
    #print(len(data), len(cubewl))
    bgsub_cube = data - bgcube
    # save or return result:
    if outfile:
        hdu[1].data = bgsub_cube
        hdu[1].header['comment'] = 'Expected Background Subtracted'
        hdu.writeto(outfile, overwrite=True)
        hdu.close()
    else:
        hdu.close()
        return bgsub_cube

Run the `jwst_backgrounds` tool before running this final cell, so that the background files exist.  We recommend using a naming convention that makes sense for you -- for us, we have named the files "[target name]_background.txt".

In [ ]:
if target == 'SGAS1723':
    expbg_file = f'path/to/jwst/expected_bg/{target}_background.txt'
    g395cube = os.path.join(output_path, folders_L3[0], lev3asnname+'_g395h-f290lp_s3d.fits')
    g140cube = os.path.join(output_path, folders_L3[0], lev3asnname+'_g140h-f100lp_s3d.fits')
    g395out = g395cube.replace('NOBG','BGSUB')
    g140out = g140cube.replace('NOBG','BGSUB')
    exp_bg_sub(g140cube, expbg_file, g140out)
    exp_bg_sub(g395cube, expbg_file, g395out)
    
if target == 'SGAS1226':
    expbg_file = f'path/to/jwst/expected_bg/{target}_background.txt'
    g235cube = os.path.join(output_path, folders_L3[0], lev3asnname+'_g235h-f170lp_s3d.fits')
    g235out = g235cube.replace('NOBG', 'BGSUB')
    exp_bg_sub(g235cube, expbg_file, g235out)
    
    
if target == 'SPT0418-47':
    expbg_file = f'path/to/jwst/expected_bg/{target}_background.txt'
    g395cube = os.path.join(output_path, folders_L3[0], lev3asnname+'_g395m-f290lp_s3d.fits')
    g395out = g395cube.replace('NOBG', 'BGSUB')
    exp_bg_sub(g395cube, expbg_file, g395out)
    

if target == 'SPT2147-50':
    expbg_file = f'path/to/jwst/expected_bg/{target}_background.txt'
    g395cube = os.path.join(output_path, folders_L3[0], lev3asnname+'_g395m-f290lp_s3d.fits')
    g395out = g395cube.replace('NOBG', 'BGSUB')
    exp_bg_sub(g395cube, expbg_file, g395out)

### allllllll done

pat yourself on the back, good job team!